In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import matplotlib.pyplot as plt
from IPython.display import clear_output
plt.plot([0,1], [0,1])
plt.show()
clear_output()

In [3]:
import torch
from torchvision import transforms, datasets, models
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
from utils.helper import set_seed
from matplotlib import pyplot as plt
from utils.xray_plot import draw_pred
import os
import re
import random
from matplotlib import pyplot as plt
import json
import cv2
#import skimage
import numpy as np
from collections import OrderedDict
import torchvision

device = "cuda" if torch.cuda.is_available() else "cpu"
import torch

In [4]:
transform_pipeline =  transforms.Compose([
     lambda x: x/255,
                               transforms.Normalize(
                                    mean=[0.485, 0.456, 0.406],
                                    std=[0.229, 0.224, 0.225])
                                    ])

In [5]:
##? 
def resnet34(outshape):
    resnet = models.resnet34(pretrained=False)
    num_features = resnet.fc.in_features
    resnet.fc = nn.Linear(num_features, outshape)
    return resnet
output_size=13
pretrained_model = resnet34(output_size)

class XrayPretrained(nn.Module):
    def __init__(self):
        super(XrayPretrained, self).__init__()
        self.resnet = pretrained_model # output size = resnet18_shape

    def forward(self, x):
        x = self.resnet(x)
        return x
    
model = XrayPretrained()    
model.load_state_dict(torch.load("041305-resnet34-6000ep.pth", map_location=torch.device('cpu')))


c:\Users\akame\AppData\Local\Programs\Python\Python310\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\akame\AppData\Local\Programs\Python\Python310\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


<All keys matched successfully>

In [7]:
# mean=[0.485, 0.456, 0.406],
#                                     std=[0.229, 0.224, 0.225])
invTrans = transforms.Compose([ transforms.Normalize(mean = [ 0., 0., 0. ],
                                                     std = [ 1/0.229, 1/0.224, 1/0.225 ]),
                                transforms.Normalize(mean = [ -0.485, -0.456, -0.406 ],
                                                     std = [ 1., 1., 1. ]),
                                lambda x: x*255
                               ])



In [8]:
# run inference on unlabeled images
test_dir = "data_unann_padded"
test_images = os.listdir(test_dir)[:20]

In [9]:
def extract_img(img_name):
    sample_image = os.path.join(test_dir, img_name)
    print(sample_image)
    img = cv2.imread(sample_image)
    if img is None:
        print("Error: Image not loaded. Check the file path.")
    resized_img = cv2.resize(img, (256,256))
    img = torch.from_numpy(resized_img).permute(2, 1, 0)
    img = transform_pipeline(img)
    return img
unlabeled_x = torch.stack( [extract_img(n) for n in test_images]) 

data_unann_padded\1014.postopAxillary.jpg
data_unann_padded\1014.preopAxillary.jpg
data_unann_padded\1023_axillary postop.png
data_unann_padded\1027_axillary post op.png
data_unann_padded\1027_axillary pre op.png
data_unann_padded\1031_ax post op.png
data_unann_padded\1031_ax pre op.png
data_unann_padded\1034_ax post op.png
data_unann_padded\1036.postopAxillary.jpg
data_unann_padded\1036.preopAxillary.jpg
data_unann_padded\1037.2ndpostopAxillary.jpg
data_unann_padded\1037.postopAxillary.jpg
data_unann_padded\1037.preopAxillary.jpg
data_unann_padded\1046.postopAxillary.jpg
data_unann_padded\1046.preopAxillary.jpg
data_unann_padded\1048.preopAx.jpg
data_unann_padded\1049_ax post op.png
data_unann_padded\1049_ax pre op.png
data_unann_padded\1050_ax pre op.png
data_unann_padded\1051.postopAxillary.jpg


In [10]:
unlabeled_x.shape

torch.Size([20, 3, 256, 256])

In [11]:
model.eval()

with torch.no_grad():
  # Get the predictions from the model
    outputs = model(unlabeled_x)


In [12]:
inv_unlabeled_x = invTrans(unlabeled_x)

In [13]:
idx = 0

In [34]:
plt.figure(figsize=(20, 8))
draw_pred(inv_unlabeled_x, outputs, idx)
idx+=1

IndexError: index 20 is out of bounds for dimension 0 with size 20

<Figure size 2000x800 with 0 Axes>